In [51]:
import pandas as pd
import numpy as np

In [52]:
df = pd.read_csv('jobs.csv')
df

,role,company_name,company_location,company_rating,salary,job_type,description
0,Data Analyst II - Trust & Safety - job post,Pinterest,"San Francisco, CA•Remote",3.8 out of 5 stars,NaN,NaN,\n\nAbout Pinterest:\n Millions of people acro...
1,Data Scientist – Learning Analytics - job post,MIT,"600 Memorial Dr Ste 1, Cambridge, MA 02139",4.2 out of 5 stars,"$110,000 - $125,000 a year",- Full-time,"\n\n\nDATA SCIENTIST, LEARNING ANALYTICS, MIT ..."
2,Data Analyst - job post,"GitHub, Inc.",Remote,4.1 out of 5 stars,"$73,500 - $195,000 a year",- Full-time,\n About GitHub: As the global home for all de...
3,Missouri Online Data Analyst I - Remote and Hy...,University of Missouri,Missouri•Remote,4.1 out of 5 stars,"$60,000 - $69,000 a year",- Full-time,\n\n\nHiring Department\n\n\n\n\n\neLearning\n...
4,Data Analyst - job post,Everside Health,Remote,3 out of 5 stars,"$75,000 - $85,000 a year",- Full-time,\n\n ABOUT THE JOB\n \n\n The Data Analyst ...
...,...,...,...,...,...,...,...
1058,Data Analyst - job post,Truepill,Remote,2.6 out of 5 stars,"$90,000 - $130,000 a year",- Full-time,"\nDescription \nAt Truepill, we power the futu..."
1059,Senior Data Analyst - job post,ServiceNow,"San Diego, CA 92121•Remote",3.8 out of 5 stars,"$84,870 - $140,130 a year",- Full-time,"\nCompany Description\n At ServiceNow, our tec..."
1060,Health Data Analyst - job post,Blue Cross and Blue Shield of Nebraska (BCBSNE),"Omaha, NE•Hybrid remote",3.9 out of 5 stars,NaN,NaN,\nBlue Cross and Blue Shield of Nebraska (BCBS...
1061,Data Scientist - job post,Key Cyber Solutions,"Richmond, VA",NaN,NaN,Part-time,\nCompany Description\n Key Cyber Solutions (K...


In [53]:
def salary_to_min_max(salary: str):
    """
    Possible cases:
    $X a year/an hour
    $X - $Y a year/an hour
    $X - $Y a month
    Up to $X a year

    if given a single value, assume $0 - $X
    if given 'Up to $X', assume $0 - $X
    """
    salary = salary.lower()
    salary = salary.replace('$', '')
    salary = salary.replace(',', '')
    min_salary = max_salary = 0.0

    if 'up to' in salary:
        salary = salary.replace('up to', '')

    if 'year' in salary:
        min_salary, max_salary = get_min_max(salary, 'a year', 1)
    elif 'hour' in salary:
        min_salary, max_salary = get_min_max(salary, 'an hour', 40 * 52)
    elif 'month' in salary:
        min_salary, max_salary = get_min_max(salary, 'a month', 12)

    return min_salary, max_salary



def get_min_max(salary: str, replacement: str, multiplication: int):
    min_salary = max_salary = 0.0

    salary = salary.replace(replacement, '')
    salary = salary.strip()
    split = salary.split(' - ')

    # if range is given
    if len(split) == 2:
        min_salary, max_salary = split
        min_salary = float(min_salary) * multiplication
        max_salary = float(max_salary) * multiplication
    # if only a single value is given
    else:
        max_salary = float(split[0]) * multiplication

    return min_salary, max_salary



def split_location(location: str):
    city = state = None
    parts = location.split(', ')
    if len(parts) == 3:
        city = parts[1]
        state = parts[2][:2]
    elif len(parts) == 2:
        city = parts[0]
        state = parts[1][:2]
    else:
        city = parts[0]
    return city, state

In [54]:
df['role'] = df['role'].str.replace('- job post', '')

df['job_location'] = np.where(df['company_location'] == 'Remote', 'Remote', 'On-site')
df['job_location'] = np.where(df['company_location'].str.contains('•'), df['company_location'].str.split('•').str[1], df['job_location'])

df['company_location'] = np.where(df['company_location'] == 'Remote', '', df['company_location'])
df['company_location'] = df['company_location'].str.split('•').str[0]

df[['company_location', 'company_state']] = df['company_location'].apply(lambda x: pd.Series(split_location(x)))
df['company_state'].fillna('', inplace=True)

df['company_rating'] = df['company_rating'].str.split(' out of ').str[0]
df['company_rating'] = df['company_rating'].fillna(0)
df['company_rating'] = pd.to_numeric(df['company_rating'])

df['salary'] = df['salary'].fillna('0')
df[['min_salary', 'max_salary']] = df['salary'].apply(lambda x: pd.Series(salary_to_min_max(x)))
df.drop('salary', axis=1, inplace=True)

df['job_type'] = df['job_type'].str.lstrip(' -')
df['job_type'] = df['job_type'].fillna('')

df['description'] = df['description'].str.strip()

df

,role,company_name,company_location,company_rating,job_type,description,job_location,company_state,min_salary,max_salary
0,Data Analyst II - Trust & Safety,Pinterest,San Francisco,3.8,,About Pinterest:\n Millions of people across t...,Remote,CA,0.0,0.0
1,Data Scientist – Learning Analytics,MIT,Cambridge,4.2,Full-time,"DATA SCIENTIST, LEARNING ANALYTICS, MIT Open L...",On-site,MA,110000.0,125000.0
2,Data Analyst,"GitHub, Inc.",,4.1,Full-time,About GitHub: As the global home for all devel...,Remote,,73500.0,195000.0
3,Missouri Online Data Analyst I - Remote and Hy...,University of Missouri,Missouri,4.1,Full-time,Hiring Department\n\n\n\n\n\neLearning\n\n\n\n...,Remote,,60000.0,69000.0
4,Data Analyst,Everside Health,,3.0,Full-time,ABOUT THE JOB\n \n\n The Data Analyst is res...,Remote,,75000.0,85000.0
...,...,...,...,...,...,...,...,...,...,...
1058,Data Analyst,Truepill,,2.6,Full-time,"Description \nAt Truepill, we power the future...",Remote,,90000.0,130000.0
1059,Senior Data Analyst,ServiceNow,San Diego,3.8,Full-time,"Company Description\n At ServiceNow, our techn...",Remote,CA,84870.0,140130.0
1060,Health Data Analyst,Blue Cross and Blue Shield of Nebraska (BCBSNE),Omaha,3.9,,Blue Cross and Blue Shield of Nebraska (BCBSNE...,Hybrid remote,NE,0.0,0.0
1061,Data Scientist,Key Cyber Solutions,Richmond,0.0,Part-time,Company Description\n Key Cyber Solutions (KCS...,On-site,VA,0.0,0.0


In [55]:
df.to_csv('preprocessed.csv', index=False)